In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset
from surprise import Reader
import math
from surprise import SVD, SVDpp
from surprise.model_selection import cross_validate
from surprise import accuracy
from collections import defaultdict
from surprise import  NMF, BaselineOnly, KNNBasic
from lightfm import LightFM
from surprise.model_selection import GridSearchCV
from lightfm.data import Dataset as LFMDataset
from lightfm.evaluation import precision_at_k, recall_at_k
import random


In [ ]:
# load data
ratings_data = pd.read_csv('mooc_data/Data/mooc.train.rating', sep='\t', names=['user', 'item', 'rating'])
test = pd.read_csv('mooc_data/Data/mooc.test.rating', sep='\t', names=['user', 'item', 'rating'])

reader = Reader(rating_scale=(0, 1))
train_data = Dataset.load_from_df(ratings_data[['user', 'item', 'rating']], reader)
trainset = train_data.build_full_trainset()
test_data = Dataset.load_from_df(test[['user', 'item', 'rating']], reader)
testset = test_data.build_full_trainset().build_testset()


all_users = set(ratings_data['user']).union(set(test['user']))
all_items = set(ratings_data['item']).union(set(test['item']))


def hit_and_ndcg_per_user(predicted_items, actual_items):
    hits = 0
    ndcg = 0.0
    for rank, item in enumerate(predicted_items):
        if item in actual_items:
            hits += 1  
            ndcg += 1 / np.log2(rank + 2) 
    return hits, ndcg


def evaluate_user(recommended_items, actual_items):
    hr = False
    hr_count = 0
    dcg = 0.0
    idcg = 0.0
    precision = 0.0
    recall = 0.0
    ndcg = 0.0
    
    if (len(actual_items) == 0):
        return {"Precision": precision, "Recall": recall, "HR": hr_count, "NDCG": ndcg}
    

    for rank, item in enumerate(recommended_items, start=1):
        if item in actual_items:
            hr = True
            precision += 1 / len(recommended_items)
            recall += 1 / len(actual_items)
            dcg += 1 / np.log2(rank + 2)  # rank+2 because log2(1) is 0 and ranks start at 0

        # Calculate IDCG part
        if rank <= len(actual_items):
            idcg += 1 / np.log2(rank + 2)  # Same as DCG for the 'ideal' case
    if hr == True:
        hr_count += 1
    
    # Calculate nDCG for the user
    ndcg = dcg / idcg if idcg > 0 else 0

    return {"Precision": precision, "Recall": recall, "HR": hr_count, "NDCG": ndcg}


In [ ]:
# Random

# Number of recommendations
N = 5

# Generate random recommendations for each user
recommendations = {user: random.sample(all_items, N) for user in all_users}

# Initialize evaluation metrics
recall, hr_count, ndcg_score, precision = 0, 0, 0, 0

# Evaluate recommendations
for user in all_users:
    actual_items = test[test['user'] == user]['item'].tolist()
    predicted_items = recommendations.get(user, [])
    metrics = evaluate_user(predicted_items, actual_items)
    hr_count += metrics['HR'] > 0
    ndcg_score += metrics['NDCG']
    recall += metrics['Recall']
    precision += metrics['Precision']

# Compute average metrics
num_users = len(all_users)
hr = hr_count / num_users
ndcg = ndcg_score / num_users
recall = recall / num_users
precision = precision / num_users

# Output results
print(f"HR@{N}: {hr}")
print(f"nDCG@{N}: {ndcg}")
print(f"Recall@{N}: {recall}")
print(f"Precision@{N}: {precision}")


In [ ]:
# Popular

# 计算物品流行度
item_popularity = ratings_data['item'].value_counts()

# Top N
N = 5
top_items = item_popularity.head(N).index.tolist()

# 为每个用户生成相同的推荐列表
recommendations = {user: top_items for user in test_data['user'].unique()}

# 应用评估函数
recall, hr_count, ndcg_score, precision = 0, 0, 0, 0
for user in test_data['user'].unique():
    actual_items = test_data[test_data['user'] == user]['item'].tolist()
    predicted_items = recommendations.get(user, [])
    metrics = evaluate_user(predicted_items, actual_items)
    hr_count += metrics['HR'] > 0
    ndcg_score += metrics['NDCG']
    recall += metrics['Recall']
    precision += metrics['Precision']

# 计算平均指标
num_users = len(test_data['user'].unique())
hr = hr_count / num_users
ndcg = ndcg_score / num_users
recall = recall / num_users
precision = precision / num_users

print(num_users)

# 输出结果
print(f"HR@10: {hr}")
print(f"nDCG@10: {ndcg}")
print(f"Recall@10: {recall}")
print(f"Precision@10: {precision}")

In [ ]:
# NMF

# 定义要调优的参数
param_grid = {'n_factors': [15, 20, 25],
              'n_epochs': [50, 100, 30]}  # PMF实现，即SVD无偏置项

# 使用网格搜索进行交叉验证
gs = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=3)

# 在数据集上找到最佳参数
gs.fit(train_data)

# 最佳RMSE得分
print(gs.best_score['rmse'])

# 最佳参数
print(gs.best_params['rmse'])

# 使用最佳参数训练NMF模型
trainset = train_data.build_full_trainset()
nmf = NMF(n_factors=gs.best_params['rmse']['n_factors'],
           n_epochs=gs.best_params['rmse']['n_epochs'])
# # 初始化NMF模型
# nmf = NMF()
# 训练模型
nmf.fit(trainset)
# 进行预测
predictions = nmf.test(testset)


# 生成推荐列表
recommendations = defaultdict(list)
for user in all_users:
    for item in all_items:
        est = nmf.predict(user, item).est
        recommendations[user].append((item, est))
    recommendations[user] = sorted(recommendations[user], key=lambda x: x[1], reverse=True)[:10]

recall = 0
hr_count = 0
ndcg_score = 0
for user in all_users:
    actual_items = test[test['user'] == user]['item'].tolist()
    predicted_items = [x[0] for x in recommendations[user]]
    metrics = evaluate_user(predicted_items, actual_items)
    hr_count += metrics['HR'] > 0
    ndcg_score += metrics['NDCG']
    recall += metrics['Recall']
    precision += metrics['Precision']

hr = hr_count / len(test['user'].unique())
ndcg = ndcg_score / len(test['user'].unique())
recall = recall/ len(test['user'].unique())

precision = precision/ len(test['user'].unique())

print(f"Precision@10: {precision}")
print(f"HR@5: {hr}")
print(f"nDCG@5: {ndcg}")
print(f"recall@5: {recall}")

In [ ]:
# PMF

# 定义要调优的参数
param_grid = {'n_factors': [50, 100, 150],
              'n_epochs': [20,30,50],
              'lr_all': [0.002],
              'reg_all': [0.02],
              'biased': [False]}  # PMF实现，即SVD无偏置项

# 使用网格搜索进行交叉验证
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

# 在数据集上找到最佳参数
gs.fit(train_data)

# 最佳RMSE得分
print(gs.best_score['rmse'])

# 最佳参数
print(gs.best_params['rmse'])

# 使用最佳参数训练PMF模型
trainset = train_data.build_full_trainset()
pmf = SVD(n_factors=gs.best_params['rmse']['n_factors'],
           n_epochs=gs.best_params['rmse']['n_epochs'],
           lr_all=gs.best_params['rmse']['lr_all'],
           reg_all=gs.best_params['rmse']['reg_all'],
           biased=False)
pmf.fit(trainset)

# 进行预测
predictions = pmf.test(testset)
# 生成推荐列表
recommendations = defaultdict(list)
for user in all_users:
    for item in all_items:
        est = pmf.predict(user, item).est
        recommendations[user].append((item, est))
    recommendations[user] = sorted(recommendations[user], key=lambda x: x[1], reverse=True)[:10]

recall = 0
hr_count = 0
ndcg_score = 0
for user in all_users:
    actual_items = test[test['user'] == user]['item'].tolist()
    predicted_items = [x[0] for x in recommendations[user]]
    metrics = evaluate_user(predicted_items, actual_items)
    hr_count += metrics['HR'] > 0
    ndcg_score += metrics['NDCG']
    recall += metrics['Recall']

hr = hr_count / len(test['user'].unique())
ndcg = ndcg_score / len(test['user'].unique())
recall = recall/ len(test['user'].unique())


print(f"HR@10: {hr}")
print(f"nDCG@10: {ndcg}")
print(f"recall@10: {recall}")

In [ ]:
# KNN (User + Item)

# 初始化KNNBasic模型
# 可以选择不同的相似度度量方法和配置
sim_options = {  # 可以选择 'cosine', 'msd', 'pearson', 'pearson_baseline'
    'user_based': False  # False 表示itemKNN，True 表示userKNN
}
knn = KNNBasic(sim_options=sim_options)

# 训练模型
knn.fit(trainset)

# 进行预测
predictions = knn.test(testset)

# 生成推荐列表
recommendations = defaultdict(list)
for user in all_users:
    for item in all_items:
        est = knn.predict(user, item).est
        recommendations[user].append((item, est))
    recommendations[user] = sorted(recommendations[user], key=lambda x: x[1], reverse=True)[:10]

recall = 0
hr_count = 0
ndcg_score = 0
for user in all_users:
    actual_items = test[test['user'] == user]['item'].tolist()
    predicted_items = [x[0] for x in recommendations[user]]
    metrics = evaluate_user(predicted_items, actual_items)
    hr_count += metrics['HR'] > 0
    ndcg_score += metrics['NDCG']
    recall += metrics['Recall']
    precision += metrics['Precision']

hr = hr_count / len(test['user'].unique())
ndcg = ndcg_score / len(test['user'].unique())
recall = recall/ len(test['user'].unique())
precision = precision/ len(test['user'].unique())

print(f"Precision@10: {precision}")
print(f"HR@5: {hr}")
print(f"nDCG@5: {ndcg}")
print(f"recall@5: {recall}")

In [ ]:
# LightFM + BPR/WARP

# 准备数据集
lfm_dataset = LFMDataset()
# 同时传入训练集和测试集中的所有用户和物品
all_users = set(ratings_data['user']).union(set(test['user']))
all_items = set(ratings_data['item']).union(set(test['item']))
lfm_dataset.fit(users=(x for x in all_users),
                items=(x for x in all_items))

# print(lfm_dataset)

# 构建交互矩阵
(interactions, _) = lfm_dataset.build_interactions(((row['user'], row['item']) for index, row in ratings_data.iterrows()))

# print(interactions)

# 训练BPR模型
# loss = 'warp'
model = LightFM(loss='bpr')



model.fit(interactions, epochs=10)



# 构建测试集
(test_interactions, _) = lfm_dataset.build_interactions(((row['user'], row['item']) for index, row in test.iterrows()))




# 生成推荐列表
num_users, num_items = lfm_dataset.interactions_shape()
recommendations = defaultdict(list)
for user_id in range(num_users):
    scores = model.predict(user_id, np.arange(num_items))
    top_items = np.argsort(-scores)[:5]
    recommendations[user_id] = top_items

# 计算评价指标
recall = 0
hr_count = 0
ndcg_score = 0
precision = 0
for user_id in range(num_users):
    actual_items = test[test['user'] == user_id]['item'].tolist()
    predicted_items = recommendations[user_id]
    metrics = evaluate_user(predicted_items, actual_items)
    hr_count += metrics['HR'] > 0
    ndcg_score += metrics['NDCG']
    recall += metrics['Recall']
    precision += metrics['Precision']

hr = hr_count / len(test['user'].unique())
ndcg = ndcg_score / len(test['user'].unique())
recall = recall / len(test['user'].unique())
precision = precision / len(test['user'].unique())

# 输出结果
print(f"Recall@5: {recall}")
print(f"Precision@5: {precision}")
print(f"HR@5: {hr}")
print(f"nDCG@5: {ndcg}")